# Data Engineering 2: Graded Lab 02
---------------

#### Grading
For this graded lab you can get a total of 20 points. These 20 points count 10% of your final grade for the course.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab02.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

#### Part 01: In this part of the graded lab you need to solve different tasks for analysing Twitter data (10 points)
###### Note: The data for this part is contained in the part01 folder

##### Task 01: Print the top10 words of the tweets (2 points)
###### Read the 'text' column of the tweets from the file "2023_01_01" into an rdd and print the top10 words of the tweets. Note: lowercase all words and remove the stopwords from the stopwords list of the archive.

In [0]:
from pyspark.sql.functions import lower, explode, split, count, col, regexp_replace, length, trim

# Read the JSON file
tweets_2023_01_01 = spark.read.json("/FileStore/tables/part01/data_january2023/2023_01_01.json")

# Read stopwords from file and collect as a Python list
stopwords_df = spark.read.text("/FileStore/tables/part01//stopwords.txt")
stopwords = [row.value.strip() for row in stopwords_df.collect()]

# Remove URLs
tweets_2023_01_01 = tweets_2023_01_01.withColumn(
    "text",
    regexp_replace(col("text"), "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "")
)

# Lowercase the text
tweets_2023_01_01 = tweets_2023_01_01.withColumn("text", lower(col("text")))

# Split text into words, remove punctuation, and explode into rows
words_df = tweets_2023_01_01.select(
    explode(
        split(
            regexp_replace(col("text"), r"[^a-zA-Z0-9]", " "),  # Replace non-alphanumeric with space
            r"\s+"
        )
    ).alias("word")
)

# Remove empty strings and stopwords
words_df = words_df.withColumn("word", trim(col("word")))
words_df = words_df.filter(
    (length(col("word")) > 0) &
    (~col("word").isin(stopwords))
)

# Count word frequencies and show top 10
print("\nTop 10 most frequent words:")
words_df.groupBy("word").count().orderBy(col("count").desc()).show(10)


Top 10 most frequent words:
+--------+-----+
|    word|count|
+--------+-----+
|    year|  638|
|   happy|  449|
|    2023|  292|
|      rt|  169|
| wishing|  145|
|  family|   98|
| healthy|   85|
|      qt|   68|
|    work|   67|
|congress|   66|
+--------+-----+
only showing top 10 rows



##### Task 02: Find the user with the most tweets in January 2023 (2 points) 
###### Use paired RDDs and their functions to find the user with the most tweets in January 2023.

In [0]:
# Read all JSON files as a DataFrame first
tweets_df = spark.read.json("/FileStore/tables/part01/data_january2023/*.json")

# Convert the DataFrame to an RDD
tweets_rdd = tweets_df.rdd

# Map to ((user_id, screen_name), 1) pairs
user_screen_pairs = tweets_rdd.map(lambda row: ((row.user_id, row.screen_name), 1))

# Reduce by key to count tweets per (user_id, screen_name)
user_screen_counts = user_screen_pairs.reduceByKey(lambda a, b: a + b)

# Find the (user_id, screen_name) with the most tweets
most_tweets_user = user_screen_counts.reduce(lambda a, b: a if a[1] > b[1] else b)

# Print the result
print("\nUser with the most tweets in January 2023:")
print(f"User ID: {most_tweets_user[0][0]}, Screen Name: {most_tweets_user[0][1]}, Number of tweets: {most_tweets_user[1]}")


User with the most tweets in January 2023:
User ID: 14344823, Screen Name: SenateGOP, Number of tweets: 750


##### Task 03: Print the top5 users with the most tweets in January 2023 including their top 5 terms (2 points)
###### Use paired RDDs and their functions to find the users with the most tweets in January 2023. Afterwards analyse their text content of the tweets and print the top 5 terms from all their posted tweets. Note: lowercase all words and remove the stopwords from the stopwords list of the archive.

In [0]:
import re

# Map to ((user_id, screen_name), 1) pairs
user_screen_pairs = tweets_rdd.map(lambda row: ((row.user_id, row.screen_name), 1))

# Reduce by key to count tweets per user
user_screen_counts = user_screen_pairs.reduceByKey(lambda a, b: a + b)

# Get the top 5 users with the most tweets
top5_users = user_screen_counts.takeOrdered(5, key=lambda x: -x[1])

# Read stopwords from file
stopwords = set([row.value.strip().lower() for row in spark.read.text("/FileStore/tables/part01/stopwords.txt").collect()])

# Remove stopwords and URL's
def extract_words(text):
    # Remove URL's
    text = re.sub(r"http[s]?://\S+", "", text)
    # Lowercase and split into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Remove stopwords
    return [word for word in words if word not in stopwords]

for user in top5_users:
    user_id, screen_name = user[0]
    # Filter tweets for this user
    user_tweets = tweets_rdd.filter(lambda row: row.user_id == user_id)
    # Extract and clean words from all their tweets
    words_rdd = user_tweets.flatMap(lambda row: extract_words(row.text))
    # Count word frequencies
    word_counts = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
    # Get top 5 terms
    top5_terms = word_counts.takeOrdered(5, key=lambda x: -x[1])
    # Print results
    print(f"\nTop 5 terms for User ID: {user_id}, Screen Name: {screen_name}:")
    for word, count in top5_terms:
        print(f"{word}: {count}")


Top 5 terms for User ID: 14344823, Screen Name: SenateGOP:
rt: 648
border: 436
biden: 307
crisis: 120
amp: 118

Top 5 terms for User ID: 1247612029, Screen Name: RitchieTorres:
rt: 506
ritchietorres: 459
santos: 411
george: 329
house: 300

Top 5 terms for User ID: 377609596, Screen Name: ericswalwell:
rt: 581
qt: 211
mccarthy: 147
ericswalwell: 129
house: 116

Top 5 terms for User ID: 1137600571, Screen Name: RepCloakroom:
rep: 285
amendment: 275
debate: 120
vote: 108
begun: 101

Top 5 terms for User ID: 15207668, Screen Name: HouseGOP:
rt: 332
biden: 206
house: 171
housegop: 121
republicans: 117


##### Task 04: Print minutewise the top5 terms within the tweets of the first of January (2 points)
###### Find a solution of your choice to print the top5 terms within the tweets of the file "2023_01_01". Note: lowercase all words and remove the stopwords from the stopwords list of the archive.

In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, split, explode, trim, length
from pyspark.sql import Window
import pyspark.sql.functions as F

# Read tweets and stopwords
tweets_df = spark.read.json("/FileStore/tables/part01/data_january2023/2023_01_01.json")
stopwords = [row.value.strip().lower() for row in spark.read.text("/FileStore/tables/part01/stopwords.txt").collect()]

# Remove URLs from text
tweets_df = tweets_df.withColumn(
    "clean_text",
    regexp_replace(col("text"), r"http[s]?://\S+", "")
)

# Lowercase and split into words, remove punctuation
tweets_df = tweets_df.withColumn(
    "clean_text",
    lower(col("clean_text"))
)
tweets_df = tweets_df.withColumn(
    "word",
    explode(split(regexp_replace(col("clean_text"), r"[^a-zA-Z0-9]", " "), r"\s+"))
)

# Remove empty strings and stopwords
tweets_df = tweets_df.withColumn("word", trim(col("word")))
tweets_df = tweets_df.filter(
    (length(col("word")) > 0) &
    (~col("word").isin(stopwords))
)

# Extract minute from time
tweets_df = tweets_df.withColumn("minute", col("time").substr(1, 16))  # "2023-01-01T00:00"

# Count word frequencies per minute
word_counts = tweets_df.groupBy("minute", "word").count()

# For each minute, get the top 5 words
window = Window.partitionBy("minute").orderBy(col("count").desc())
word_counts = word_counts.withColumn("rank", F.row_number().over(window))
top5_per_minute = word_counts.filter(col("rank") <= 5)

# Now, for each minute, create a DataFrame and store in a dictionary
minutes = [row.minute for row in top5_per_minute.select("minute").distinct().collect()]
minute_dfs = {}

for minute in minutes:
    df = top5_per_minute.filter(col("minute") == minute).select("minute", "word", "count")
    minute_dfs[minute] = df

# Print the top5 words in tweets of every minute
for minute in sorted(minute_dfs.keys()):
    print(f"\nTop 5 words for minute {minute}:")
    minute_dfs[minute].show()


Top 5 words for minute 2023-01-01T00:00:
+----------------+-------+-----+
|          minute|   word|count|
+----------------+-------+-----+
|2023-01-01T00:00|   year|   24|
|2023-01-01T00:00|  happy|   21|
|2023-01-01T00:00|   2023|    8|
|2023-01-01T00:00|wishing|    6|
|2023-01-01T00:00|healthy|    4|
+----------------+-------+-----+


Top 5 words for minute 2023-01-01T00:01:
+----------------+------+-----+
|          minute|  word|count|
+----------------+------+-----+
|2023-01-01T00:01|  year|   11|
|2023-01-01T00:01| happy|    8|
|2023-01-01T00:01|  2023|    4|
|2023-01-01T00:01|growth|    2|
|2023-01-01T00:01| peace|    2|
+----------------+------+-----+


Top 5 words for minute 2023-01-01T00:02:
+----------------+-----+-----+
|          minute| word|count|
+----------------+-----+-----+
|2023-01-01T00:02|happy|    1|
|2023-01-01T00:02| year|    1|
+----------------+-----+-----+


Top 5 words for minute 2023-01-01T00:03:
+----------------+-------+-----+
|          minute|   word

##### Task 05: Your analysis (2 points)
###### Find an interesting analysis question for the Twitter data and answer it with a solution of your choice. Explain shortly, why your question is interesting and what the value of your created information is.

In [0]:
# Timezones

offset_to_region = {
    # North and South America (West to East)
    "-10:00": "Hawaii-Aleutian Time",
    "-09:00": "Alaska Time",
    "-08:00": "Pacific Time (US/Canada)",
    "-07:00": "Mountain Time (US/Canada)",
    "-06:00": "Central Time (US/Canada)",
    "-05:00": "Eastern Time (US/Canada)",
    "-04:00": "Atlantic Time (Canada)/Caribbean",
    "-03:00": "Argentina/Brazil/Chile",
    "-02:00": "Mid-Atlantic",

    # Europe and Africa (West to East)
    "+00:00": "UK/Ireland/Portugal (GMT)",
    "+01:00": "Central European Time (Germany, France, Italy)",
    "+02:00": "Eastern European Time (Finland, Greece, Egypt)",
    "+03:00": "Moscow, East Africa",

    # Asia and Oceania (West to East)
    "+04:00": "Dubai, UAE, Azerbaijan",
    "+05:00": "Pakistan, Kazakhstan",
    "+05:30": "India, Sri Lanka",
    "+06:00": "Bangladesh, Bhutan",
    "+07:00": "Thailand, Vietnam, Indonesia",
    "+08:00": "China, Singapore, Malaysia",
    "+09:00": "Japan, South Korea",
    "+09:30": "Central Australia",
    "+10:00": "Eastern Australia",
    "+11:00": "Solomon Islands",
    "+12:00": "New Zealand, Fiji"
}

In [0]:
def analyze_user_timezone(user_id_input):
    """
    Analysiert die Zeitzonen-Aktivität eines spezifischen Twitter-Users.

    Args:
        user_id_input (str): Die Twitter User ID

    Returns:
        None: Druckt die Analyse-Ergebnisse
    """

    try:
        # Lade alle Tweets aus Januar 2023
        tweets_df = spark.read.json("/FileStore/tables/part01/data_january2023/*.json")
        tweets_rdd = tweets_df.rdd

        # Filtere Tweets des spezifischen Users
        user_tweets = tweets_rdd.filter(lambda row: row.user_id == user_id_input)

        # Prüfe ob User existiert
        if user_tweets.count() == 0:
            print(f"Keine Tweets gefunden für User ID: {user_id_input}")
            return

        # Hole Screen Name des Users
        screen_name = user_tweets.first().screen_name

        print(f"\nZeitzonenanalyse für User {screen_name} (ID: {user_id_input}):")
        print(f"Gesamtzahl Tweets: {user_tweets.count()}")

        # Extrahiere Timezone Offsets
        def extract_offset(row):
            import re
            match = re.search(r'([+-]\d{2}:\d{2})$', row.time)
            return match.group(1) if match else None

        offsets = user_tweets.map(extract_offset).filter(lambda x: x is not None)

        # Zähle Häufigkeit der Timezones
        offset_counts = offsets.map(lambda offset: (offset, 1)).reduceByKey(lambda a, b: a + b)

        # Finde häufigste Timezone
        if not offset_counts.isEmpty():
            most_common_offset = offset_counts.reduce(lambda a, b: a if a[1] > b[1] else b)

            print(f"\nHäufigster Timezone Offset: {most_common_offset[0]}")
            print(f"Anzahl Tweets in dieser Timezone: {most_common_offset[1]}")
            print(f"Wahrscheinliche Region: {offset_to_region.get(most_common_offset[0], 'Unbekannte Region')}")

            # Zeige Verteilung aller Zeitzonen
            print("\nVerteilung aller Zeitzonen:")
            for offset, count in sorted(offset_counts.collect()):
                region = offset_to_region.get(offset, "Unbekannte Region")
                percentage = (count / user_tweets.count()) * 100
                print(f"Timezone {offset} ({region}): {count} Tweets ({percentage:.1f}%)")
        else:
            print("Keine Timezone-Informationen gefunden in den Tweets.")

    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {str(e)}")

# Beispiel-Nutzung:
analyze_user_timezone(most_tweets_user[0][0])


Zeitzonenanalyse für User SenateGOP (ID: 14344823):
Gesamtzahl Tweets: 750

Häufigster Timezone Offset: -05:00
Anzahl Tweets in dieser Timezone: 750
Wahrscheinliche Region: Eastern Time (US/Canada)

Verteilung aller Zeitzonen:
Timezone -05:00 (Eastern Time (US/Canada)): 750 Tweets (100.0%)


#### Part 02: In this part of the graded lab you need to solve different tasks for analysing Graph data (10 points)

###### Note: The data for this part is contained in the part02 folder. The library to import (like mentioned in Lab06 is also in the archive)

##### Task 01: Construct the graph (2 points)
###### Create a graph with the structure you can find on the images in the part02 folder. Print ten items of the vertices and 10 of the edges.
###### Note:  There is one image for vertices and one image for edges. You need to do some transformation to get to the desired result.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id, trim
from graphframes import GraphFrame

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Flight Graph") \
    .getOrCreate()

# Read CSV files
airports_file = '/FileStore/tables/part02/airports.csv'
flight_delays_file = '/FileStore/tables/part02/flight_delays.csv'
routes_file = '/FileStore/tables/part02/routes.csv'

airports_df = spark.read.csv(airports_file, header=True)
flight_delays_df = spark.read.csv(flight_delays_file, header=True)
routes_df = spark.read.csv(routes_file, header=True)

# Join airports with routes to get Airline ID for each airport
# (Assuming 'Source airport ID' in routes matches 'Airport ID' in airports)
vertices = airports_df.join(
    routes_df,
    airports_df["Airport ID"] == routes_df["Source airport ID"],
    "left"
).select(
    col("Airport ID").alias("airportId"),
    col("Name").alias("name"),
    col("City").alias("city"),
    col("Country").alias("country"),
    col("IATA").alias("id"),
    col("ICAO").alias("icao"),
    col("Latitude").alias("lat"),
    col("Longitude").alias("long"),
    col("Altitude").alias("altitude"),
    col("Timezone").alias("timezone"),
    col("DST").alias("dst"),
    col("Tz database time zone").alias("tz_database_time_zone"),
    col("Type").alias("type"),
    col("Source").alias("source")
).dropDuplicates(["id"])  # Remove duplicates if any

# Create edges DataFrame
edges = flight_delays_df.select(
    monotonically_increasing_id().alias("tripid"),
    col("ORIGIN").alias("src"),
    col("DEST").alias("dst"),
    col("ORIGIN_CITY_NAME").alias("origin_city_name"),
    col("DEST_CITY_NAME").alias("dest_city_name"),
    col("ARR_DELAY").alias("delay")
)

# Create the GraphFrame
g = GraphFrame(vertices, edges)

## Note

What we noticed here is that the Airline ID for the port in the example: 
`143` does not correspond with the Airline ID we are getting `\\N` so we will assume, that the airline ID in the Image should be the Airport ID, instead of the Airline ID. 

With this approach we can get the required row showcased in the Image provided for the vertices.

In [0]:
display(g.vertices.filter(g.vertices["airportId"] == 143))

airportId,name,city,country,id,icao,lat,long,altitude,timezone,dst,tz_database_time_zone,type,source
143,Timmins/Victor M. Power,Timmins,Canada,YTS,CYTS,48.56969833,-81.37670136,967,-5,A,America/Toronto,airport,OurAirports


In [0]:
display(g.edges.head(9))

tripid,src,dst,origin_city_name,dest_city_name,delay
0,DFW,LAS,"Dallas/Fort Worth, TX","Las Vegas, NV",-6.00
1,DFW,LAS,"Dallas/Fort Worth, TX","Las Vegas, NV",-5.00
2,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",-16.00
3,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",-2.00
4,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",-8.00
5,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",-1.00
6,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",0.00
7,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",2.00
8,ORD,DFW,"Chicago, IL","Dallas/Fort Worth, TX",-16.00


##### Task 02: Motifs (2 points)
###### Define a pattern which detects the all flights from New York with destination as Las Vegas and total delay of the flight should be no more than 60 minutes.

In [0]:
motifs = g.find("(a)-[e]->(b)")

result = motifs.filter(
    (col("a.city").like("%New York%")) &
    (col("b.city").like("%Las Vegas%")) &
    (col("e.delay").cast("float") <= 60)
)

display(result.select(
    col("a.id").alias("New York Airport ID"), col("a.name").alias("New York Airport Name"), col("b.id").alias("Las Vegas Airport ID"), col("b.name").alias("Las Vegas Airport Name"), "e.tripid", "e.delay"
))

/databricks/spark/python/pyspark/sql/dataframe.py:149: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


New York Airport ID,New York Airport Name,Las Vegas Airport ID,Las Vegas Airport Name,tripid,delay
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16825,14.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16826,15.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16827,20.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16828,9.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16829,35.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16830,11.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16835,38.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16836,28.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16837,36.00
JFK,John F Kennedy International Airport,LAS,McCarran International Airport,16838,26.00


##### Task 03: Graph Pattern Analysis (2 points)
###### What are the flight routes with no direct connection?

Option 1:

In [0]:
from pyspark.sql.functions import col, broadcast

# 1. Find all two-leg connections: a -> m -> b
motifs = g.find("(a)-[e1]->(m); (m)-[e2]->(b)") \
    .filter(
        (col("a.id") != col("b.id")) &
        (col("a.id") != col("m.id")) &
        (col("m.id") != col("b.id"))
    ) \
    .select(
        col("a.id").alias("src"),
        col("b.id").alias("dst")
    ).dropDuplicates()

# 2. Get all direct connections
direct = g.edges.select(
    col("src"),
    col("dst")
).dropDuplicates()

# 3. Exclude those with a direct connection (broadcast if direct is small)
no_direct = motifs.join(
    broadcast(direct),
    on=["src", "dst"],
    how="left_anti"
)

# 4. (Optional) Limit for testing
no_direct.limit(100).show()

Option 2:

In [0]:
from pyspark.sql.functions import col

# 1. Find all two-leg connections: A -> M -> B
two_leg = edges.alias("e1") \
    .join(
        edges.alias("e2"),
        col("e1.dst") == col("e2.src")
    ) \
    .select(
        col("e1.src").alias("src"),
        col("e2.dst").alias("dst"),
        col("e1.dst").alias("mid")
    ) \
    .filter(
        (col("src") != col("dst")) &
        (col("src") != col("mid")) &
        (col("mid") != col("dst"))
    ).dropDuplicates(["src", "dst"])

# 2. Get all direct connections
direct = edges.select("src", "dst").dropDuplicates()

# 3. Exclude those with a direct connection
no_direct = two_leg.join(
    direct,
    on=["src", "dst"],
    how="left_anti"
)

# 4. Show the result
no_direct.show()

Both options generate a huge amount of data because of the cartesian Product used to get the lwo-leg connections.

For Option 1 it was around: 2'000 ExaBytes

For Option 2 it was around 25 TB

This is not really feasable in this scenario. So I will limit the number of airports in the solution below. If one wants to run the above options feel free :D.

In [0]:
small_airports = vertices.filter(col("city").isin("New York", "Las Vegas", "Dallas"))
small_airports_sel = small_airports.select("id")
small_edges = edges.join(small_airports_sel, edges.src == small_airports_sel.id, "inner").select("tripid", "src", "dst", "origin_city_name", "dest_city_name", "delay")
g_small = GraphFrame(small_airports, small_edges)

In [0]:
display(g_small.edges)

tripid,src,dst,origin_city_name,dest_city_name,delay
21,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-1.00
22,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",26.00
23,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-27.00
24,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",42.00
25,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-13.00
26,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",69.00
27,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-3.00
28,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",4.00
29,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-1.00
30,LAS,DFW,"Las Vegas, NV","Dallas/Fort Worth, TX",-22.00


In [0]:
# Now run the motif search on this much smaller graph
motifs = g_small.find("(a)-[e1]->(m); (m)-[e2]->(b)") \
    .filter(
        (col("a.id") != col("b.id")) &
        (col("a.id") != col("m.id")) &
        (col("m.id") != col("b.id"))
    ) \
    .select(
        col("a.id").alias("src"),
        col("b.id").alias("dst")
    ).dropDuplicates()

motifs.limit(10).show()

+---+---+
|src|dst|
+---+---+
|LGA|LAS|
|JFK|DAL|
|LAS|LGA|
|DAL|JFK|
+---+---+



##### Task 04: What are the most important airports, according to PageRank? (2 points)
###### Note: Your allowed to use the pageRank function of the graph library.

This also has quite a while to compute...

But it can finish it in about 10 minutes.

In [0]:
# Run PageRank on your graph
results = g.pageRank(maxIter=10)

# The resulting GraphFrame has two DataFrames: vertices and edges with PageRank scores
# Show the top 10 most important airports by PageRank
display(results.vertices.select("id", "name", "city", "pagerank")
    .orderBy(col("pagerank").desc())
    .limit(10))

id,name,city,pagerank
ATL,Hartsfield Jackson Atlanta International Airport,Atlanta,102.2259908228889
ORD,Chicago O'Hare International Airport,Chicago,60.95055586237836
DFW,Dallas Fort Worth International Airport,Dallas-Fort Worth,53.55321323215381
DEN,Denver International Airport,Denver,51.83808442869418
LAX,Los Angeles International Airport,Los Angeles,46.11118872302073
SFO,San Francisco International Airport,San Francisco,38.35802055618077
PHX,Phoenix Sky Harbor International Airport,Phoenix,37.98369154459288
MSP,Minneapolis-St Paul International/Wold-Chamberlain Airport,Minneapolis,36.4414244335698
IAH,George Bush Intercontinental Houston Airport,Houston,34.68377381142953
DTW,Detroit Metropolitan Wayne County Airport,Detroit,34.42893377222441


##### Task 05: Your analysis (2 points)
###### Find an interesting analysis question for the Graph data and answer it with a solution of your choice. Explain shortly, why your question is interesting and what the value of your created information is.

As we are interested in this and have some time we will do the following few analyses:
1. What are the top Airports by Number of Connections (Degree Centrality)?
2. What is the average Arrival Delay by Airport?
3. What are the most Common Flight Routes?
4. How many Flights per country are there?

We will explain why this question is interesting for each Topic seperatly below

##### Top Airpotrs (Degree Centrality)

Could be intzeresting for analysis from the point of view from a Flight bureau's point of view. To make more flights if delays in this region are high for example.

In [0]:
# Out-degree: Number of outgoing flights from each airport
out_degree = g.edges.groupBy("src").count().orderBy(col("count").desc())
display(out_degree.limit(10))

src,count
ATL,30138
ORD,18782
LAX,17314
DEN,17030
DFW,15304
SFO,13283
PHX,13257
LAS,12487
MCO,11007
IAH,10805


In [0]:
# In-degree: Number of incoming flights to each airport
in_degree = g.edges.groupBy("dst").count().orderBy(col("count").desc())
display(in_degree.limit(10))

dst,count
ATL,30135
ORD,18783
LAX,17315
DEN,17021
DFW,15324
SFO,13289
PHX,13250
LAS,12487
MCO,10989
IAH,10788


##### Average arrival delay

This could be interesting to know for a tourist or a plan inthustiast to know where not to go to expect good arrival times. This could also correlate with the weather, but his is not the topic here.

In [0]:
# Average delay for each destination airport
avg_delay = g.edges.groupBy("dst").agg({"delay": "avg"}).withColumnRenamed("avg(delay)", "avg_delay")
display(avg_delay.orderBy(col("avg_delay").desc()).limit(10))

dst,avg_delay
ELM,81.76923076923077
BPT,67.0
GGG,61.833333333333336
BMI,38.96330275229358
LAW,35.68
ABI,35.407407407407405
LWS,29.653061224489797
GRB,27.11387900355872
ESC,25.86046511627907
ACT,25.252525252525253


##### Common flight routes

This could be interesting for someone like booking.com or airscanner. To give ideas on where to travel based on these trends.

In [0]:
# Count number of flights for each route
route_counts = edges.groupBy("src", "dst").count().orderBy(col("count").desc())
display(route_counts.limit(10))

src,dst,count
SFO,LAX,1286
LAX,SFO,1248
LAX,JFK,1047
JFK,LAX,1046
LAX,LAS,978
LAS,LAX,971
HNL,OGG,786
OGG,HNL,786
SEA,LAX,777
LAX,SEA,777


##### Flights per country

This could be used for a statistical departement in the government to work on reducing the CO2 footprint of the country by reducing in-land flights for example.

In [0]:
# Join edges (flights) with vertices (airports) to get the origin country
flights_with_origin_country = edges.join(
    vertices.select(col("id").alias("airport_id"), "country"),
    edges.src == col("airport_id"),
    "left"
)

# Group by country and count flights
flights_per_origin_country = flights_with_origin_country.groupBy("country").count().orderBy(col("count").desc())

display(flights_per_origin_country.limit(10))

country,count
United States,446710
Puerto Rico,2658
Virgin Islands,608
Guam,31
American Samoa,10


In [0]:
# Join edges (flights) with vertices (airports) to get the destination country
flights_with_dest_country = edges.join(
    vertices.select(col("id").alias("airport_id"), "country"),
    edges.dst == col("airport_id"),
    "left"
)

# Group by country and count flights
flights_per_dest_country = flights_with_dest_country.groupBy("country").count().orderBy(col("count").desc())

display(flights_per_dest_country.limit(10))

country,count
United States,446729
Puerto Rico,2640
Virgin Islands,607
Guam,31
American Samoa,10
